In [1]:
"""
Genetic Algorithm Demonstration
==============================

This program demonstrates how genetic algorithms work by evolving a population
of random strings to match a target phrase.

Genetic algorithms are optimization algorithms inspired by natural selection:
1. Start with a random population
2. Evaluate fitness of each individual
3. Select the fittest individuals for reproduction
4. Create offspring through crossover and mutation
5. Repeat until solution is found
"""

import random
import string

class GeneticAlgorithm:
    def __init__(self, target, population_size=1000, mutation_rate=0.01, elite_size=20):
        """
        Initialize the genetic algorithm.
        
        Args:
            target: The target string we want to evolve
            population_size: Number of individuals in each generation
            mutation_rate: Probability of mutation for each gene
            elite_size: Number of best individuals to carry over to next generation
        """
        self.target = target
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.genes = string.ascii_letters + ' .,!?;:\'"'  # Possible characters
        
    def create_individual(self):
        """Create a random individual (string) of the same length as target."""
        return ''.join(random.choice(self.genes) for _ in range(len(self.target)))
    
    def create_population(self):
        """Create initial random population."""
        return [self.create_individual() for _ in range(self.population_size)]
    
    def fitness(self, individual):
        """
        Calculate fitness score for an individual.
        Higher score = better fitness (more similar to target).
        """
        score = 0
        for i in range(len(self.target)):
            if individual[i] == self.target[i]:
                score += 1
        return score / len(self.target)  # Normalize to 0-1
    
    def selection(self, population, scores):
        """
        Select individuals for breeding using tournament selection.
        Better fitness = higher chance of selection.
        """
        selected = []
        for _ in range(len(population)):
            # Tournament selection: pick 3 random individuals, choose the best
            tournament = random.sample(list(zip(population, scores)), 3)
            winner = max(tournament, key=lambda x: x[1])
            selected.append(winner[0])
        return selected
    
    def crossover(self, parent1, parent2):
        """
        Create offspring by combining genes from two parents.
        Uses single-point crossover.
        """
        crossover_point = random.randint(1, len(self.target) - 1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        return child
    
    def mutate(self, individual):
        """
        Randomly mutate genes based on mutation rate.
        This introduces genetic diversity.
        """
        individual = list(individual)
        for i in range(len(individual)):
            if random.random() < self.mutation_rate:
                individual[i] = random.choice(self.genes)
        return ''.join(individual)
    
    def evolve_population(self, population):
        """Create next generation through selection, crossover, and mutation."""
        # Calculate fitness scores
        scores = [self.fitness(ind) for ind in population]
        
        # Keep elite individuals (best performers)
        population_with_scores = list(zip(population, scores))
        population_with_scores.sort(key=lambda x: x[1], reverse=True)
        elite = [ind for ind, _ in population_with_scores[:self.elite_size]]
        
        # Select parents for breeding
        selected = self.selection(population, scores)
        
        # Create new generation
        children = elite.copy()  # Start with elite
        
        while len(children) < self.population_size:
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            
            child = self.crossover(parent1, parent2)
            child = self.mutate(child)
            children.append(child)
        
        return children[:self.population_size]
    
    def run(self, max_generations=1000):
        """Run the genetic algorithm until target is found or max generations reached."""
        population = self.create_population()
        generation = 0
        
        print(f"Target: '{self.target}'")
        print(f"Population size: {self.population_size}")
        print(f"Mutation rate: {self.mutation_rate}")
        print("\nStarting evolution...\n")
        
        while generation < max_generations:
            # Evaluate current generation
            scores = [self.fitness(ind) for ind in population]
            best_index = scores.index(max(scores))
            best_individual = population[best_index]
            best_score = scores[best_index]
            
            # Print progress every 10 generations
            if generation % 10 == 0:
                avg_fitness = sum(scores) / len(scores)
                print(f"Generation {generation:4d} | "
                      f"Best: '{best_individual}' | "
                      f"Fitness: {best_score:.3f} | "
                      f"Avg: {avg_fitness:.3f}")
            
            # Check if we found the target
            if best_individual == self.target:
                print(f"\n🎉 Target found in generation {generation}!")
                print(f"Final result: '{best_individual}'")
                return best_individual, generation
            
            # Evolve to next generation
            population = self.evolve_population(population)
            generation += 1
        
        # If we didn't find exact match
        scores = [self.fitness(ind) for ind in population]
        best_index = scores.index(max(scores))
        best_individual = population[best_index]
        
        print(f"\nMax generations reached. Best result: '{best_individual}'")
        return best_individual, generation


# Example: Evolving different target strings
def demonstrate_examples():
    """Run multiple examples to show the genetic algorithm in action."""
    
    examples = [
        "Hello World!",
        "Genetic Algorithm",
        "Evolution works!",
        "Python is awesome"
    ]
    
    print("="*60)
    print("GENETIC ALGORITHM DEMONSTRATIONS")
    print("="*60)
    
    for target in examples:
        print(f"\n{'='*60}")
        print(f"EVOLVING: '{target}'")
        print(f"{'='*60}")
        
        # Create and run genetic algorithm
        ga = GeneticAlgorithm(
            target=target,
            population_size=500,
            mutation_rate=0.01,
            elite_size=20
        )
        
        result, generations = ga.run(max_generations=5000)
        
        print(f"\nCompleted in {generations} generations")
        print("-"*60)


# Advanced example: Optimizing a mathematical function
def function_optimization_example():
    """
    Demonstrate using genetic algorithm to find maximum of a function.
    Finding maximum of f(x) = -x^2 + 5x + 10 in range [0, 10]
    """
    print("\n" + "="*60)
    print("FUNCTION OPTIMIZATION EXAMPLE")
    print("="*60)
    print("Finding maximum of f(x) = -x² + 5x + 10 in range [0, 10]")
    
    class FunctionGA:
        def __init__(self, population_size=50, mutation_rate=0.1):
            self.population_size = population_size
            self.mutation_rate = mutation_rate
            self.min_x = 0
            self.max_x = 10
            
        def create_individual(self):
            """Create random x value in range."""
            return random.uniform(self.min_x, self.max_x)
        
        def fitness(self, x):
            """Calculate f(x) = -x² + 5x + 10"""
            return -x**2 + 5*x + 10
        
        def mutate(self, x):
            """Add small random change to x."""
            if random.random() < self.mutation_rate:
                mutation = random.gauss(0, 0.5)
                x = x + mutation
                x = max(self.min_x, min(self.max_x, x))  # Keep in bounds
            return x
        
        def crossover(self, parent1, parent2):
            """Average crossover."""
            return (parent1 + parent2) / 2
        
        def run(self, generations=50):
            """Run the optimization."""
            population = [self.create_individual() for _ in range(self.population_size)]
            
            for gen in range(generations):
                # Evaluate fitness
                fitness_scores = [self.fitness(x) for x in population]
                
                # Sort by fitness
                paired = list(zip(population, fitness_scores))
                paired.sort(key=lambda x: x[1], reverse=True)
                
                # Print best every 10 generations
                if gen % 10 == 0:
                    best_x = paired[0][0]
                    best_fitness = paired[0][1]
                    print(f"Generation {gen:3d} | Best x: {best_x:.4f} | f(x): {best_fitness:.4f}")
                
                # Create next generation
                elite = [x for x, _ in paired[:10]]  # Keep top 10
                new_population = elite.copy()
                
                while len(new_population) < self.population_size:
                    parent1 = random.choice([x for x, _ in paired[:25]])
                    parent2 = random.choice([x for x, _ in paired[:25]])
                    child = self.crossover(parent1, parent2)
                    child = self.mutate(child)
                    new_population.append(child)
                
                population = new_population
            
            # Final result
            fitness_scores = [self.fitness(x) for x in population]
            best_idx = fitness_scores.index(max(fitness_scores))
            best_x = population[best_idx]
            
            print(f"\nOptimization complete!")
            print(f"Best x found: {best_x:.4f}")
            print(f"Maximum value: {self.fitness(best_x):.4f}")
            print(f"(Theoretical maximum at x=2.5: {self.fitness(2.5):.4f})")
    
    # Run the optimization
    optimizer = FunctionGA()
    optimizer.run()


if __name__ == "__main__":
    # Run string evolution examples
    demonstrate_examples()
    
    # Run function optimization example
    function_optimization_example()
    
    print("\n" + "="*60)
    print("GENETIC ALGORITHM CONCEPTS SUMMARY")
    print("="*60)
    print("""
    1. INITIALIZATION: Create random population
    2. EVALUATION: Calculate fitness of each individual
    3. SELECTION: Choose parents based on fitness
    4. CROSSOVER: Combine parents to create offspring
    5. MUTATION: Randomly modify genes for diversity
    6. REPLACEMENT: Form new generation
    7. REPEAT until solution found or max generations reached
    
    Key Parameters:
    - Population Size: Larger = more diversity but slower
    - Mutation Rate: Balance between exploration and exploitation
    - Elite Size: Preserve best solutions across generations
    - Selection Method: How to choose parents (tournament, roulette, etc.)
    """)

GENETIC ALGORITHM DEMONSTRATIONS

EVOLVING: 'Hello World!'
Target: 'Hello World!'
Population size: 500
Mutation rate: 0.01

Starting evolution...

Generation    0 | Best: 'iUpwL Zt'l m' | Fitness: 0.167 | Avg: 0.016
Generation   10 | Best: 'HYllo Worrd!' | Fitness: 0.833 | Avg: 0.608

🎉 Target found in generation 14!
Final result: 'Hello World!'

Completed in 14 generations
------------------------------------------------------------

EVOLVING: 'Genetic Algorithm'
Target: 'Genetic Algorithm'
Population size: 500
Mutation rate: 0.01

Starting evolution...

Generation    0 | Best: 'GOBsiQMMlijqViHcz' | Fitness: 0.118 | Avg: 0.016
Generation   10 | Best: 'GenecIc lB;or"trm' | Fitness: 0.588 | Avg: 0.462
Generation   20 | Best: 'Genetic Avgorithm' | Fitness: 0.941 | Avg: 0.868
Generation   30 | Best: 'Genetic Avgorithm' | Fitness: 0.941 | Avg: 0.931
Generation   40 | Best: 'Genetic Avgorithm' | Fitness: 0.941 | Avg: 0.933
Generation   50 | Best: 'Genetic Avgorithm' | Fitness: 0.941 | Avg: 